# Checking Kaustav's conjecture using gradient descent

In this notebook we check a certain conjecture in matrix theory, which we refer to as Kaustav's conjecture, after the colleague who formulated it. The conjecture arises from the problem of maximizing orbital correlations in so-called fermionic 'free' quantum states over unitary rotations of the underlying orbital space.

## 1. The conjecture



## 2. Numerically testing the conjecture

## 3. The gradient descent method

## 4. Import libraries and custom modules

First, we import the necessary libraries and our custom conjugate gradient implementation.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add the src directory to the path to import our custom module
sys.path.insert(0, os.path.join('..', 'src'))

# Import our functions
# from kaustav_conj.core import ...
from kaustav_conj.utils import h

In [8]:
# testing
print(h( 1 - 1.e-16))


0.0


## 5. Testing the conjecture

Test for different values of d, \lambda, \textbf{n}, U_0. For actual sampling you'll probably need cluster, when it comes to getting actual results (if any are needed)